In [2]:
from keras import layers
import keras
import keras.backend as K
from matplotlib import pyplot as plt
from pylab import rcParams
from keras.layers import Activation, Dense,Dot,merge, Dropout,Lambda,Permute,Multiply,LSTM, Conv2D, Flatten, MaxPooling2D, LSTM,RepeatVector,Reshape,TimeDistributed,UpSampling1D
# import common as com
import keras.models
from keras import backend as K
from keras.layers import Input, Dense, BatchNormalization, Activation
from keras.models import Model
# from keras.layers.wrappers import TimeDistributed

In [3]:
# Models

In [4]:
width = 128
height = 64
channels = 1
input_dim = 640
timesteps = 10
n_features = 64
lr = 0.001
batch_size = 1

In [5]:
import keras.backend as K
import numpy as np

numpyA = np.ones([1,64])

def multA(x):
    A = K.variable(np.ones([1,64]))

    return K.dot(x,A)
# def multa(x)
def multA_T(x):
    A = K.variable(np.ones([1,10]))

    return K.dot(x,A)
# weight_2 = Lambda(lambda x:x*0.2)
# weight_gru1 = weight_2(gru1)
# output = Lambda(lambda x: K.sum(x, axis=-1))(vec)
# model.add(Lambda(multA))

In [6]:
#-------------------------------------------#
#   注意力模块
#-------------------------------------------#
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, lstm_units)

    # (batch_size, time_steps, lstm_units) -> (batch_size, lstm_units, time_steps)
    a = Permute((2, 1))(inputs)

    # 对最后一维进行全连接
    # (batch_size, lstm_units, time_steps) -> (batch_size, lstm_units, time_steps)
    a = Dense(10, activation='softmax')(a)

    # (batch_size, lstm_units, time_steps) -> (batch_size, time_steps, lstm_units)
    a_probs = Permute((2, 1), name='attention_vec')(a)

    # 相乘
    # 相当于获得每一个step中，每个维度在所有step中的权重
    output_attention_mul = Multiply()([inputs,a_probs])
#     output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul


## Generators Encoder


In [7]:
import tensorflow as tf
input_layer = Input(name='input', shape=(input_dim, ))

input_layer1 = Reshape((timesteps,64))(input_layer)#[10,64]
att = TimeDistributed(Dense(64))(input_layer1)#[B,T,F]
att = Activation('sigmoid')(att)

att1 = Lambda(lambda x: K.sum(x, axis=-1))(att)
att1 = Reshape([-1,1])(att1)
att1 = Lambda(multA)(att1)
att1 = Reshape([-1,64])(att1)

att = Lambda(lambda x:x*64)(att)
att1 = Lambda(lambda x:1/x)(att1)
freq_fnorm = Multiply()([att,att1])

h = Reshape((640,))(freq_fnorm)
# h= Flatten()(cnn_input)
# print(h.shape)
h = Dense(512)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(256)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(256)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(128)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(128)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(8)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

  
g_e = keras.models.Model(inputs=input_layer, outputs=h)


g_e.summary()





Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 640)          0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 10, 64)       0           input[0][0]                      
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 10, 64)       4160        reshape_1[0][0]                  
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 10, 64)       0           time_distributed_1[0][0]         
________________________________________________________________________________________

## Generator

In [8]:
input_layer = Input(name='input', shape=(input_dim,))

x = g_e(input_layer)


y = Dense(128)(x)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(128)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(256)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(256)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(512)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(input_dim)(y)


g = keras.models.Model(inputs=input_layer, outputs=y)

g.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 640)               0         
_________________________________________________________________
model_1 (Model)              (None, 8)                 585064    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               1152      
_________________________________________________________________
batch_normalization_7 (Batch (None, 128)               512       
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_8 (Batch (None, 128)               512 

## Encoder

In [9]:

input_layer = Input(name='input', shape=(input_dim, ))

input_layer1 = Reshape((timesteps,64))(input_layer)#[10,64]
att = TimeDistributed(Dense(64))(input_layer1)#[B,T,F]
att = Activation('sigmoid')(att)

att1 = Lambda(lambda x: K.sum(x, axis=-1))(att)
att1 = Reshape([-1,1])(att1)
att1 = Lambda(multA)(att1)
att1 = Reshape([-1,64])(att1)

att = Lambda(lambda x:x*64)(att)
att1 = Lambda(lambda x:1/x)(att1)
freq_fnorm = Multiply()([att,att1])
cnn_input = Multiply()([input_layer1,freq_fnorm])

z = Reshape((640,))(cnn_input)

z = Dense(512)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(256)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(256)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(128)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(128)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(8)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

encoder = keras.models.Model(input_layer, z)
encoder.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 640)          0                                            
__________________________________________________________________________________________________
reshape_5 (Reshape)             (None, 10, 64)       0           input[0][0]                      
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 10, 64)       4160        reshape_5[0][0]                  
__________________________________________________________________________________________________
activation_13 (Activation)      (None, 10, 64)       0           time_distributed_2[0][0]         
____________________________________________________________________________________________

## feature extractor

In [10]:
input_layer = Input(name='input', shape=(input_dim,))

f = Dense(512)(input_layer)
f = BatchNormalization()(f)
f = Activation('relu')(f)

h = Dense(256)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

f = Dense(256)(f)
f = BatchNormalization()(f)
f = Activation('relu')(f)

f = Dense(128)(f)
f = BatchNormalization()(f)
f = Activation('relu')(f)

f = Dense(128)(f)
f = BatchNormalization()(f)
f = Activation('relu')(f)

feature_extractor = keras.models.Model(input_layer, f)

feature_extractor.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 640)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               328192    
_________________________________________________________________
batch_normalization_18 (Batc (None, 512)               2048      
_________________________________________________________________
activation_20 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               131328    
_________________________________________________________________
batch_normalization_20 (Batc (None, 256)               1024      
_________________________________________________________________
activation_22 (Activation)   (None, 256)               0   

## gan trainer

In [13]:
class AdvLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AdvLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori_feature = feature_extractor(x[0])
        gan_feature = feature_extractor(x[1])
        return K.mean(K.square(ori_feature - K.mean(gan_feature, axis=0)))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)
    
class CntLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CntLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori = x[0]
        gan = x[1]
        return K.mean(K.square(ori - gan))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)
    
class EncLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(EncLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori = x[0]
        gan = x[1]
        return K.mean(K.square(g_e(ori) - encoder(gan)))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)
    
# model for training
input_layer = layers.Input(name='input', shape=(input_dim,))
gan = g(input_layer) # g(x)

adv_loss = AdvLoss(name='adv_loss')([input_layer, gan])
cnt_loss = CntLoss(name='cnt_loss')([input_layer, gan])
enc_loss = EncLoss(name='enc_loss')([input_layer, gan])

gan_trainer = keras.models.Model(input_layer, [adv_loss, cnt_loss, enc_loss])

# loss function
def loss(yt, yp):
    return yp

losses = {
    'adv_loss': loss,
    'cnt_loss': loss,
    'enc_loss': loss,
}

lossWeights = {'cnt_loss': 10.0, 'adv_loss': 1.0, 'enc_loss': 1.0}

# compile
gan_trainer.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=losses, loss_weights=lossWeights)

In [14]:
gan_trainer.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 640)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 640)          1166568     input[0][0]                      
__________________________________________________________________________________________________
adv_loss (AdvLoss)              [(None, 640), (None, 0           input[0][0]                      
                                                                 model_2[2][0]                    
__________________________________________________________________________________________________
cnt_loss (CntLoss)              [(None, 640), (None, 0           input[0][0]                

## discriminator

In [15]:
input_layer = layers.Input(name='input', shape=(input_dim,))

f = feature_extractor(input_layer)

d = Dense(32)(f)
d = BatchNormalization()(d)
d = Activation('elu')(d)

d = layers.Dense(1, activation='sigmoid', name='d_out')(d)    

d = keras.models.Model(input_layer, d)
d.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 640)               0         
_________________________________________________________________
model_4 (Model)              (None, 128)               513024    
_________________________________________________________________
dense_26 (Dense)             (None, 32)                4128      
_________________________________________________________________
batch_normalization_23 (Batc (None, 32)                128       
_________________________________________________________________
activation_25 (Activation)   (None, 32)                0         
_________________________________________________________________
d_out (Dense)                (None, 1)                 33        
Total params: 517,313
Trainable params: 515,201
Non-trainable params: 2,112
_________________________________________________

In [16]:
d.compile(optimizer=keras.optimizers.Adam(lr=lr), loss='binary_crossentropy')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Load Data

In [18]:
# from keras.datasets import mnist
# import cv2
import numpy as np
import sys
import os
# additional
import numpy as np
import librosa
from tqdm import tqdm
import librosa.core
import librosa.feature
n_mels=64
n_frames= 10
n_hop_frames= 1
n_fft= 1024
hop_length= 512
power= 2.0
in_dir_c = '/root/DCASE2021/dev_data/pump/train/'
wav_list_c = os.listdir(in_dir_c)
idx = 0

for  wav_file_c in tqdm(wav_list_c):     
    if wav_file_c.endswith('.wav'):
        indir_c1 = in_dir_c + wav_file_c
        dims = n_mels * n_frames
        y, sr = librosa.load(indir_c1, sr=None, mono=True)
#         y -= np.mean(y)
        mel_spectrogram = librosa.feature.melspectrogram(y=y,
                                                     sr=sr,
                                                     n_fft=n_fft,
                                                     hop_length=hop_length,
                                                     n_mels=n_mels,
                                                     power=power)
        log_mel_spectrogram = 20.0 / power * np.log10(np.maximum(mel_spectrogram, sys.float_info.epsilon))#(128, 313)
#         log_mel_spectrogram = np.abs(log_mel_spectrogram)
#         log_mel_spectrogram = np.mean(log_mel_spectrogram, axis=-1)
        
        n_vectors = len(log_mel_spectrogram[0, :]) - n_frames + 1
#         print(n_vectors)
        if n_vectors < 1:
            vectors = np.empty((0, dims))
        vectors = np.zeros((n_vectors, dims))#(250, 8192)
        for t in range(n_frames):
            vectors[:, n_mels * t : n_mels * (t + 1)] = log_mel_spectrogram[:, t : t + n_vectors].T
        vectors = vectors[: : n_hop_frames, :]
        if idx == 0:
            data = np.zeros((len(wav_list_c) * vectors.shape[0], dims), float)
        data[vectors.shape[0] * idx : vectors.shape[0] * (idx + 1), :] = vectors#(752250, 8192)
        idx = 1+idx
        
        #print(data[0].shape)
#x_ok = data.reshape(data.shape[0], 64, 128, 1)#[data.shape[0],64,128,1]
x_ok = data
print(x_ok.shape)

image = np.reshape(x_ok[i:i+1], (64, 128))
# image = image * 127 + 127
plt.imshow(image)  



in_dir_c1 = '/root/DCASE2021/dev_data/valve/source_test/'
wav_list_c1 = os.listdir(in_dir_c1)
y_test = []
idx = 0
for wav_file_c in tqdm(wav_list_c1):    
    if wav_file_c.endswith('.wav'):
#         print(wav_file_c.split('_')[4])
        if wav_file_c.split('_')[4] == 'anomaly':
            for i in range(309):
                y_test.append(0)
        else :
            for i in range(309):
                y_test.append(1)
        indir_c1 = in_dir_c1 + wav_file_c
        dims = n_mels * n_frames
        y, sr = librosa.load(indir_c1, sr=None, mono=True)
        mel_spectrogram = librosa.feature.melspectrogram(y=y,
                                                     sr=sr,
                                                     n_fft=n_fft,
                                                     hop_length=hop_length,
                                                     n_mels=n_mels,
                                                     power=power)
        log_mel_spectrogram = 20.0 / power * np.log10(np.maximum(mel_spectrogram, sys.float_info.epsilon))#(128, 313)
        n_vectors1 = len(log_mel_spectrogram[0, :]) - n_frames + 1
        
#         if n_vectors1 < 1:
#             vectors1 = np.empty((0, dims))
        vectors1 = np.zeros((n_vectors1, dims))#(250, 8192)
        for t in range(n_frames):
            vectors1[:, n_mels * t : n_mels * (t + 1)] = log_mel_spectrogram[:, t : t + n_vectors1].T
        vectors1 = vectors1[: : n_hop_frames, :]
        #print(vectors1.shape)
        if idx == 0:
            data1 = np.zeros((len(wav_list_c1) * vectors1.shape[0], dims), float)
        
        data1[vectors1.shape[0] * idx : vectors1.shape[0] * (idx + 1), :] = vectors1#(752250, 8192)
        idx = 1+idx
        
x_test = data1.reshape(data1.shape[0], 64, 128, 1)#[data.shape[0],64,128,1]
x_test = data1
x_ok.max()









100%|██████████| 6018/6018 [00:34<00:00, 173.43it/s]

(1829472, 640)


# Training

In [21]:
niter = 20000
bz = 512

In [22]:
def get_data_generator(data, batch_size=32):
    datalen = len(data)
    cnt = 0
    while True:
        idxes = np.arange(datalen)
        np.random.shuffle(idxes)
        cnt += 1
        for i in range(int(np.ceil(datalen/batch_size))):
            train_x = np.take(data, idxes[i*batch_size: (i+1) * batch_size], axis=0)
            y = np.ones(len(train_x))
            yield train_x, [y, y, y]

In [23]:
train_data_generator = get_data_generator(x_ok, bz)

In [28]:
min = 10000
for i in range(niter):
    
    
    if i % 1000 == 0:
        lr *= 0.9
        print(lr)
    # get batch x, y ###
    x, y = train_data_generator.__next__()
        
    ### train disciminator ###
    d.trainable = True
        
    fake_x = g.predict(x)
#     print(lr)    
    d_x = np.concatenate([x, fake_x], axis=0)
    d_y = np.concatenate([np.zeros(len(x)), np.ones(len(fake_x))], axis=0)
        
    d_loss = d.train_on_batch(d_x, d_y)

    ### train generator ###
    
    d.trainable = False        
    g_loss = gan_trainer.train_on_batch(x, y)
    
    if i % 100 == 0:
        if(g_loss[2]<min):
            min = g_loss[2]
            #print(type(g_loss[2]))
            print(g_loss[2],min)
            g.save("model/model_pump.hdf5")
        print(f'niter: {i+1}, g_loss: {g_loss}, d_loss: {d_loss}')

0.0004782969
6.988934 6.988934
niter: 1, g_loss: [70.4838, 0.3914475, 6.988934, 0.20300852], d_loss: 0.02509298548102379
6.821381 6.821381
niter: 101, g_loss: [68.79824, 0.3913836, 6.821381, 0.1930431], d_loss: 0.00910855457186699
niter: 201, g_loss: [69.45964, 0.3913914, 6.887008, 0.19817664], d_loss: 0.018827510997653008
niter: 301, g_loss: [71.02338, 0.38675678, 7.0421133, 0.21548603], d_loss: 0.008351164869964123
6.492843 6.492843
niter: 401, g_loss: [65.507645, 0.38961262, 6.492843, 0.18960677], d_loss: 0.00949561595916748
niter: 501, g_loss: [69.99956, 0.37988013, 6.9411664, 0.20801005], d_loss: 0.004994094371795654
niter: 601, g_loss: [66.53683, 0.37857887, 6.5945406, 0.21284321], d_loss: 0.00425156531855464
niter: 701, g_loss: [66.73599, 0.38837907, 6.614589, 0.20171762], d_loss: 0.02233564481139183
niter: 801, g_loss: [67.80025, 0.37234056, 6.7228427, 0.19948912], d_loss: 0.0197482630610466
niter: 901, g_loss: [66.17634, 0.37281024, 6.561015, 0.19337545], d_loss: 0.01156391855

niter: 8101, g_loss: [59.235, 0.33401415, 5.871101, 0.1899743], d_loss: 0.0008263522759079933
niter: 8201, g_loss: [59.852787, 0.34147722, 5.932124, 0.1900686], d_loss: 0.0014100061962381005
niter: 8301, g_loss: [60.1232, 0.3455362, 5.9580297, 0.19736578], d_loss: 0.0018269075080752373
niter: 8401, g_loss: [60.84433, 0.3422202, 6.0308676, 0.19343089], d_loss: 0.0002731106651481241


KeyboardInterrupt: 

In [ ]:
import os
import sys
import gc
########################################################################


########################################################################
# import additional libraries
########################################################################
import numpy as np
import scipy.stats
# from import
from tqdm import tqdm
try:
    from sklearn.externals import joblib
except:
    import joblib
# original lib
# import common as com

y_pred = []
start_idx = 0
n_vectors_ea_file = 304
for file_idx in range(6018):
    y_pred.append(np.mean(np.square(x_ok[start_idx : start_idx + n_vectors_ea_file, :] 
                                      - g.predict(x_ok[start_idx : start_idx + n_vectors_ea_file, :]))))
    start_idx += n_vectors_ea_file

shape_hat, loc_hat, scale_hat = scipy.stats.gamma.fit(y_pred)
gamma_params = [shape_hat, loc_hat, scale_hat]
joblib.dump(gamma_params, "model/score_distr_ToyCar.pkl")
decision_threshold = scipy.stats.gamma.ppf(q=0.9, a=shape_hat, loc=loc_hat, scale=scale_hat)        
print(decision_threshold)        
g.save("model/model_ToyTrain.hdf5")
    